In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import pi, cos, sin, sqrt, atan2
import random
from os.path import exists
from rtsvg import *
rt = RACETrack()
# Load the data
edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.edges.layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
my_df             = pl.DataFrame(_lu_)
g                 = rt.createNetworkXGraph(my_df, [('fm','to')])

In [ ]:
# Find the communities
communities       = nx.community.louvain_communities(g)
community_lu      = {}
cluster_number    = max(max(my_df['fm']),max(my_df['to'])) + 1000
for _community_ in communities:
    community_lu[cluster_number] = set(_community_)
    cluster_number += 1

In [ ]:
# Collapse the communities
my_df_communities = rt.collapseDataFrameGraphByClusters(my_df, [('fm','to')], community_lu)
g_communities     = rt.createNetworkXGraph(my_df_communities, [('__fm__','__to__')])
communities_pos   = nx.spring_layout(g_communities)

In [ ]:
# Find the bounding box and determine the coordinate transforms
x_min, y_min, x_max, y_max = rt.positionExtents(communities_pos)
w, h         = 900, 900
cd_r         = 64
x_ins, y_ins = cd_r + 10, cd_r + 10
wxToSx       = lambda x: 1.5*cd_r + 5 + (w - 3*cd_r)*(x-x_min)/(x_max-x_min)
wyToSy       = lambda y: 1.5*cd_r + 5 + (h - 3*cd_r)*(y-y_min)/(y_max-y_min)

In [ ]:
svg = [f'<svg x="0" y="0" width="{w}" height="{h}"><rect x="0" y="0" width="{w}" height="{h}" fill="white" />']
for _community_ in community_lu:
    _df_   = my_df.filter(pl.col('fm').is_in(community_lu[_community_]) | pl.col('to').is_in(community_lu[_community_]))
    _opacity_ = max(0.8 - len(community_lu[_community_])/20.0, 0.02)
    _cd_   = rt.chordDiagram(_df_, [('fm', 'to')], w=cd_r*2, h=cd_r*2, x_ins=0, y_ins=0, link_opacity=_opacity_, draw_border=False, node_h=4)
    sx, sy = wxToSx(communities_pos[_community_][0]), wyToSy(communities_pos[_community_][1])
    svg.append(f'<g transform="translate({sx-cd_r}, {sy-cd_r})">{_cd_._repr_svg_()}</g>')
svg.append('</svg>')
rt.tile([''.join(svg)])